In [14]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
from bs4 import BeautifulSoup
import requests

url = 'https://ftp.ebi.ac.uk/pub/databases/spot/eQTL/sumstats/'
uPage = uReq(url)
page_soup = BeautifulSoup(uPage.read(), "html.parser")
uPage.close()

In [3]:
studies = [p.text for p in page_soup.find_all("a")[5:]]

In [4]:
links = []  

for study in studies:
    try:
        url = 'https://ftp.ebi.ac.uk/pub/databases/spot/eQTL/sumstats/' + study + 'tx/'
        uPage = uReq(url)
        page_soup = BeautifulSoup(uPage.read(), "html.parser")
        uPage.close()
        data = [p.text for p in page_soup.find_all('a') if p.text.endswith('.all.tsv.gz')]
        if len(data) == 1:
            links.append(data)          
        else:
            data_naive = [d for d in data if 'naive' in d]
            if len(data_naive) == 0:
                links += data
            elif len(data_naive) == 1:
                links.append(data_naive)
            elif len(data_naive) > 1:
                data_naive_all = [d for d in data_naive if 'all' in d]
                if len(data_naive_all) == 0:
                    links += data_naive
                else:
                    links.append(data_naive_all)
    except Exception:
        print(study)

CEDAR/
Fairfax_2012/
Fairfax_2014/
Gilchrist_2021/
Kasela_2017/
Naranbhai_2015/


In [5]:
def flatten_list(_2d_list):
    flat_list = []
    for element in _2d_list:
        if type(element) is list:
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [6]:
tissues_list = flatten_list(links)

In [7]:
links_list = ['https://ftp.ebi.ac.uk/pub/databases/spot/eQTL/sumstats/'+studies[i]+'tx/'+tissues_list[j]
              for i in range (len(studies)) 
              for j in range (len(tissues_list)) 
              if studies[i][:-1] in tissues_list[j] ]

In [8]:
with open('links.txt', 'w') as fp:
    for item in links_list:
        fp.write(item + "\n")


In [21]:
from subprocess import call
import multiprocessing as mp

In [34]:
def load_study(path):
    call (['wget',path])
    print(path + ' downloaded')
    call(['gunzip', path.split('/')[10]])    
    print(path + ' unzipped')

In [144]:
pool_obj = mp.Pool(10)

In [53]:
pool_obj.map(load_study, links_list)

In [156]:
from os import listdir

def find_tsv_filenames( path_to_dir, suffix=".tsv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

In [157]:
loaded = find_tsv_filenames('.')

In [158]:
len(loaded)


91

In [159]:
not_loaded = []

for i in range (len(links_list)):
    check = False
    for j in range (len(loaded)):
        if (loaded[j]  in links_list[i]):
            check = True
        else:
            continue
    if (not check):
        not_loaded.append(links_list[i])

In [160]:
not_loaded

[]